In [1]:
import requests
import datetime

In [2]:
ENV = 'stg'

In [3]:
PROMPT_CONTROL_MAP = {
    'stg' : "172.18.243.211",
    'prd' : "172.27.10.166",
}

In [4]:
PROMPT_ID_MAP = {
    'stg' : '123954',
    'prd' : None,
}

In [5]:
## extract_date_time function 프롬프트
time_prompt_template = '''Current Time:: 2023-05-10 12:30:01 수요일
Expression:: 이번주 중에 뭐 할꺼 있지?
Time Range:: {"expression": "이번주", "start_time": "2023-05-08 00:00:00 월요일", "end_time": "2023-05-14 23:59:59 일요일"}

Current Time:: 2023-03-23 02:15:42 목요일
Expression:: 어제부터 오늘까지 해야 할꺼 있었는데...
Time Range:: {"expression": "어제부터 오늘까지", "start_time": "2023-03-22 00:00:00 수요일", "end_time": "2023-03-23 02:15:42 목요일"}

Current Time:: 2023-04-01 05:56:41 토요일
Expression:: 이런건 아무도 모르지~
Time Range:: {"expression": "none"}

Current Time:: 2023-12-12 05:07:25 화요일
Expression:: 이번주는 해야 할꺼 있었는데...
Time Range:: {"expression": "이번주는", "start_time": "2023-12-11 00:00:00 월요일", "end_time": "2023-12-17 23:59:59 일요일"}

Current Time:: 2023-09-25 17:42:38 월요일
Expression:: 술한잔 하려고 하는데 밤에 한 7시쯤 어때?
Time Range:: {"expression": "밤에 한 7시쯤", "start_time": "2023-09-25 19:00:00 월요일", "end_time": "2023-09-25 19:59:59 월요일"}

Current Time:: 2023-07-11 12:32:27 화요일
Expression:: 나 한... 저번주 부터 사귀기로 했어~
Time Range:: {"expression": "저번주 부터", "start_time": "2023-07-03 00:00:00 월요일", "end_time": "2023-07-11 12:32:27 화요일"}

Current Time:: 2023-01-15 21:51:17 화요일
Expression:: 차라리 이럴꺼면 휴가를 내고 해외에 한번 가봐
Time Range:: {"expression": "none"}

Current Time:: 2023-02-11 07:30:12 토요일
Expression:: 너무나도 슬프게도 화요일이 우리가 마지막으로 볼 수 있는 시간이네요 ㅠㅜ
Time Range:: {"expression": "화요일", "start_time": "2023-07-14 00:00:00 화요일", "end_time": "2023-07-14 23:59:59 화요일"}

Current Time:: 2023-12-10 07:30:12 일요일
Expression:: 저번해부터 쭉 이 일을 마무리하느라 고생하셨습니다.
Time Range:: {"expression": "저번해부터 쭉", "start_time": "2022-01-01 00:00:00 토요일", "end_time": "2023-12-10 07:30:12 일요일"}

Current Time:: 2023-06-17 07:30:12 토요일
Expression:: 오전에 시작한거 현 시점까지 마무리해야대!
Time Range:: {"expression": "오전에 시작한거 현 시점까지", "start_time": "2023-06-17 00:00:00 토요일", "end_time": "2023-06-17 07:30:12 토요일"}

Current Time:: 2023-09-06 03:44:11 수요일
Expression:: 이따 6시 40분에 택시 예약 잡아줘.
Time Range:: {"expression": "6시 40분에", "start_time": "2023-09-06 06:40:00 수요일", "end_time": "2023-09-06 06:49:59 수요일"}

Current Time:: 2024-01-01 00:07:19 월요일
Expression:: 우리 모두 올해에는 행복하세요~
Time Range:: {"expression": "올해에는", "start_time": "2024-01-01 00:00:00 월요일", "end_time": "2024-12-31 23:59:59 화요일"}

Current Time:: 2021-06-21 05:04:58 월요일
Expression:: 앞으로 3시간 뒤 까지는 비가 올 예정이다.
Time Range:: {"expression": "앞으로 3시간 뒤 까지는", "start_time": "2021-06-21 05:04:58 월요일", "end_time": "2021-06-21 08:04:58 월요일"}

Current Time:: 2023-07-09 23:32:51 일요일
Expression:: 너 마감일이 이틀 뒤까지인거 알고 있는거지? 
Time Range:: {"expression": "이틀 뒤까지인거", "start_time": "2023-07-09 23:32:51 일요일", "end_time": "2023-07-11 23:59:59 화요일"}

Current Time:: 2023-10-09 04:43:33 월요일
Expression:: 우리 다음달 말쯤에는 같이 수영이나 할래?. 
Time Range:: {"expression": "다음달 말쯤에는", "start_time": "2023-11-30 00:00:00 목요일", "end_time": "2023-11-30 23:59:59 목요일"}

Current Time:: 2023-02-28 20:26:02 화요일
Expression:: 우리 주말에는 꼭 만나서 이야기좀 하자. 
Time Range:: {"expression": "주말에는", "start_time": "2023-03-04 00:00:00 토요일", "end_time": "2023-03-05 23:59:59 일요일"}

Current Time:: 2023-05-04 22:32:18 목요일
Expression:: 이거에 대해서 세시쯤에 다시 이야기 할꺼지? 
Time Range:: {"expression": "세시쯤에", "start_time": "2023-05-05 03:00:00 금요일", "end_time": "2023-05-05 03:59:59 금요일"}

Current Time:: 2023-04-18 09:11:27 화요일
Expression:: 내일은 12시에 출근하는데, 출근 전에 볼만한 영화가 있을까?
Time Range:: {"expression": "내일은 12시에 출근하는데, 출근 전", "start_time": "2023-04-19 00:00:00 수요일", "end_time": "2023-04-19 12:00:00 수요일"}
'''

## extract_date_time function 프롬프트
time_input_prompt_template = f'''
Current Time:: {{current_sent}}
Expression:: {{query}}
Time Range:: '''


def extract_date_time(query:str):
    current_dt = datetime.datetime.today()
    days_in_korean = ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]

    current_sent = current_dt.strftime('%Y-%m-%d %H:%M:%S') + f' {days_in_korean[current_dt.weekday()]}'
    content = time_prompt_template+time_input_prompt_template.format(current_sent=current_sent, query=query)
    
    headers = {
    "Accept": "application/json",
    }

    data = {
        "promptId": PROMPT_ID_MAP[ENV],
        "requestTexts": [
            {
                "key": "content",
                "value": content,
            },
        ],
        'role':'user',
    }

    resp = requests.post(
        f"http://{PROMPT_CONTROL_MAP[ENV]}:13100/api/v1/completions",
        headers=headers,
        json=data,
    )
    resp = resp.json()
    
    return resp

In [6]:
text = "앞으로 3시간 뒤에는 뭐 할까요?"

In [7]:
extract_date_time(text)

In [2]:
headers = {
    "Accept": "application/json",
}

data = {
    "promptId": '123954',
    "requestTexts": [
        {
            "key": "content",
            "value": "안녕하세요.",
        },
    ],
    'role':'user',
}

resp = requests.post(
    "http://172.18.243.211:13100/api/v1/completions",
    headers=headers,
    json=data,
)
#resp = resp.json()

ConnectionError: HTTPConnectionPool(host='172.18.243.211', port=13100): Max retries exceeded with url: /api/v1/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x105daad60>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
print(resp)